In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import math

# get current directory
curr_dir = os.getcwd()
curr_dir

'C:\\Users\\josh.galloway\\Google Drive\\Synch\\Education\\_Northeastern University\\01_MATH 7243 Machine Learning 1\\Projects\\VIX Project'

# Import Harvard-IV-4 and Lasswell dictionaries
http://www.wjh.harvard.edu/~inquirer/spreadsheet_guide.htm 

In [29]:
'''Import Harvard-IV-4 and Lasswell dictionaries found at 
    http://www.wjh.harvard.edu/~inquirer/spreadsheet_guide.htm '''

H4dict = 'H4Lvd Dictionaries\inquireraugmented.xls'
h4df_raw = pd.read_excel(os.path.join(curr_dir, H4dict))
display(h4df_raw.info())
h4df_raw.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11789 entries, 0 to 11788
Columns: 186 entries, Entry to Defined
dtypes: object(186)
memory usage: 16.7+ MB


None

,Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,...,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othrtags,Defined
0,NaN,NaN,1915,2291,1045,557,1160,833,1902,689,...,30,193,126,175,132,25,428,368,NaN,NaN
1,A,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET ART,| article: Indefinite singular article--some o...
2,ABANDON,H4Lvd,NaN,Negativ,NaN,NaN,Ngtv,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
3,ABANDONMENT,H4,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,|
4,ABATE,H4Lvd,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
5,ABATEMENT,Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,NaN
6,ABDICATE,H4,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
7,ABHOR,H4,NaN,Negativ,NaN,NaN,NaN,Hostile,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
8,ABIDE,H4,Positiv,NaN,NaN,Affil,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
9,ABILITY,H4Lvd,Positiv,NaN,NaN,NaN,NaN,NaN,Strong,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,NaN


# Clean and Format Semantic Dictionary for this Application

In [30]:
'''Clean Dataframe'''
COLS_DROP = 14  # index of columns list where begin dropping columns
COLS_KEEP = ['Entry','Source','Positiv','Negativ','Pstv','Ngtv',
             'Strong','Weak','Active','Passive']

 # drop unwanted columns and first row cause it's just data counts
#h4df = h4df_raw.drop(columns=h4df_raw.columns[COLS_DROP:]).iloc[1:]  
# Filter out unwanted columns
h4df = h4df_raw[COLS_KEEP].iloc[1:]


# make the word the index
h4df.set_index('Entry',inplace=True) 

# drop words that don't have an entry
h4df = h4df.drop(columns='Source').dropna(axis = 0, how = 'all') 

# convert strings to 0 and 1
h4df = h4df.fillna(0)
strto1 = lambda x : 1 if x != 0 else x
for col in COLS_KEEP[2:]:
    h4df[col] = h4df[col].apply(strto1)

display(h4df.info())
display(h4df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 6536 entries, ABANDON to ZEST
Data columns (total 8 columns):
Positiv    6536 non-null int64
Negativ    6536 non-null int64
Pstv       6536 non-null int64
Ngtv       6536 non-null int64
Strong     6536 non-null int64
Weak       6536 non-null int64
Active     6536 non-null int64
Passive    6536 non-null int64
dtypes: int64(8)
memory usage: 459.6+ KB


None

,Positiv,Negativ,Pstv,Ngtv,Strong,Weak,Active,Passive
Entry,,,,,,,,
ABANDON,0,1,0,1,0,1,0,0
ABANDONMENT,0,1,0,0,0,1,0,0
ABATE,0,1,0,0,0,0,0,1
ABDICATE,0,1,0,0,0,1,0,1
ABHOR,0,1,0,0,0,0,0,1


## Build Vector of Sentiment Pairs (ex Positiv/Negativ...)

In [31]:
'''Combine Positiv/Negativ and Pstv/Ngtv'''
pn = [['Positiv','Pstv'],['Negativ','Ngtv']]  # pairs to combine
combinePN = lambda x,y: x or y
for pairs in pn:
    h4df[pairs[0]] = h4df[pairs[0]] | h4df[pairs[1]]

'''Combine in to 3 dimensional set
   Positive/Negative Strong/Weak Active/Passive with
   cooridinate set 1/-1'''
COLS_3D = ['PN','SW','AP']
pn = [['Positiv','Negativ'],['Strong','Weak'],['Active','Passive']]
for i,pair in enumerate(pn):
    h4df[COLS_3D[i]] = h4df[pair[0]] - h4df[pair[1]]

'''Build np.matrix vector from coordinate pairs'''
x = h4df[COLS_3D[0]].values
y = h4df[COLS_3D[1]].values
z = h4df[COLS_3D[2]].values
v = []
for i in range(len(x)):
    v.append(list([x[i],y[i],z[i]]))
h4df['Vect'] = v

h4df.head(10)

,Positiv,Negativ,Pstv,Ngtv,Strong,Weak,Active,Passive,PN,SW,AP,Vect
Entry,,,,,,,,,,,,
ABANDON,0,1,0,1,0,1,0,0,-1,-1,0,"[-1, -1, 0]"
ABANDONMENT,0,1,0,0,0,1,0,0,-1,-1,0,"[-1, -1, 0]"
ABATE,0,1,0,0,0,0,0,1,-1,0,-1,"[-1, 0, -1]"
ABDICATE,0,1,0,0,0,1,0,1,-1,-1,-1,"[-1, -1, -1]"
ABHOR,0,1,0,0,0,0,0,1,-1,0,-1,"[-1, 0, -1]"
ABIDE,1,0,0,0,0,0,1,0,1,0,1,"[1, 0, 1]"
ABILITY,1,0,0,0,1,0,0,0,1,1,0,"[1, 1, 0]"
ABJECT,0,1,0,0,0,1,0,1,-1,-1,-1,"[-1, -1, -1]"
ABLE,1,0,1,0,1,0,0,0,1,1,0,"[1, 1, 0]"


## Average the Scores of Duplicated Words

In [32]:
'''average duplicated words of the form word#n 
    where n is # of different entries '''

# get list of unique words
words = list(h4df.index)
duplicates = []
for word in words:
    dup = '#' in str(word)
    if dup:
        duplicates.append(word)

# get list of root words in duplicates
rw = []
for word in duplicates:
    w = word.split('#')[0]
    test = w in rw
    if not test:
        rw.append(w)

# create dictionary from root words and init 
CNT, SENT = 0,1
avgSent = {}
avgCnt = {}
for w in rw:
    avgSent[w] = [0,0,0]
    avgCnt[w] = 0

# average sentiment for each root word
for word in duplicates:
    w = word.split('#')[0]
    avgCnt[w] = avgCnt[w] + 1
    avgSent[w] = list(np.array(avgSent[w]) + np.array(h4df.loc[word]['Vect']))
        
# put in dataframe
df = pd.DataFrame(index=rw, columns=h4df.columns)
for w in rw:
    sent = list(np.ndarray.round(np.array(avgSent[w])/(avgCnt[w] + 1e-12),1))
    df.loc[w]['Vect'] = sent

# Append to original
semantic_df = pd.concat([h4df,df])

# Drop duplicates
semantic_df.drop(index=duplicates, inplace=True)
semantic_df = semantic_df['Vect']

## Display Cleaned Semantic Dictionary

In [33]:
# Print out word counts as check
print('Number of Duplicated Words in Dictionary: ',len(duplicates) - len(rw))
print('Number of Words in Un-averaged Set: ',len(h4df))
print('Number of Words in Averaged Set: ',len(semantic_df))

# Display final dataframe
semantic_df.head(10)

Number of Duplicated Words in Dictionary:  1406
Number of Words in Un-averaged Set:  6536
Number of Words in Averaged Set:  5130


ABANDON         [-1, -1, 0]
ABANDONMENT     [-1, -1, 0]
ABATE           [-1, 0, -1]
ABDICATE       [-1, -1, -1]
ABHOR           [-1, 0, -1]
ABIDE             [1, 0, 1]
ABILITY           [1, 1, 0]
ABJECT         [-1, -1, -1]
ABLE              [1, 1, 0]
ABNORMAL         [-1, 0, 0]
Name: Vect, dtype: object

In [26]:
'''Save or Retrive file for Semantic Dictionary'''
ENABLE_CELL = False   # must be true to execute save or retreive
SAVE = False           # dataframe if true retrieves if false
RETREIVE = 'saves\sematic_df20200319-134749.csv'

if ENABLE_CELL:
    if SAVE:
        semantic_df_file = 'saves\sematic_df' + datetime.now().strftime("%Y%m%d-%H%M%S") + '.csv'
        pd.DataFrame(semantic_df).to_csv(os.path.join(curr_dir, semantic_df_file))
        print('File Saved: ',semantic_df_file)
    else:
        semantic_df = pd.read_csv(os.path.join(curr_dir, RETREIVE)) 
        print('File Retrieved: ',RETREIVE)
        semantic_df.columns = ['Entry','Vect']
        semantic_df.set_index(semantic_df.columns[0],inplace=True)
        semantic_df = semantic_df.Vect
        display(semantic_df.head())

File Retrieved:  saves\sematic_df20200319-134749.csv


Entry
ABANDON         [-1, -1, 0]
ABANDONMENT     [-1, -1, 0]
ABATE           [-1, 0, -1]
ABDICATE       [-1, -1, -1]
ABHOR           [-1, 0, -1]
Name: Vect, dtype: object

# Load Headline Data

In [8]:
"""Read in ABC Headlines

https://www.kaggle.com/therohk/million-headlines 

"""
abcFile = os.path.join('data\million-headlines','abcnews-date-text.csv')
abcdf_raw = pd.read_csv(os.path.join(curr_dir, abcFile))
display(abcdf_raw.info())
display(abcdf_raw.isna().sum())
abcdf_raw.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186018 entries, 0 to 1186017
Data columns (total 2 columns):
publish_date     1186018 non-null int64
headline_text    1186018 non-null object
dtypes: int64(1), object(1)
memory usage: 18.1+ MB


None

publish_date     0
headline_text    0
dtype: int64

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
5,20030219,ambitious olsson wins triple jump
6,20030219,antic delighted with record breaking barca
7,20030219,aussie qualifier stosur wastes four memphis match
8,20030219,aust addresses un security council over iraq
9,20030219,australia is locked into war timetable opp


In [9]:
'''Read in Reddit News:

Headlines are ranked by reddit users' votes, 
and only the top 25 headlines are considered for a single date.

Label Column: "1" when DJIA Adj Close value rose or stayed as the same;
"0" when DJIA Adj Close value decreased.
'''

redditFile = os.path.join('data\stocknews','Combined_News_DJIA.csv')
redditdf_raw = pd.read_csv(os.path.join(curr_dir, redditFile))
display(redditdf_raw.info())
display(redditdf_raw.isna().sum())
redditdf_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 27 columns):
Date     1989 non-null object
Label    1989 non-null int64
Top1     1989 non-null object
Top2     1989 non-null object
Top3     1989 non-null object
Top4     1989 non-null object
Top5     1989 non-null object
Top6     1989 non-null object
Top7     1989 non-null object
Top8     1989 non-null object
Top9     1989 non-null object
Top10    1989 non-null object
Top11    1989 non-null object
Top12    1989 non-null object
Top13    1989 non-null object
Top14    1989 non-null object
Top15    1989 non-null object
Top16    1989 non-null object
Top17    1989 non-null object
Top18    1989 non-null object
Top19    1989 non-null object
Top20    1989 non-null object
Top21    1989 non-null object
Top22    1989 non-null object
Top23    1988 non-null object
Top24    1986 non-null object
Top25    1986 non-null object
dtypes: int64(1), object(26)
memory usage: 419.6+ KB


None

Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    1
Top24    3
Top25    3
dtype: int64

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

## Load VIX Data

In [10]:
"""Read in VIX Data

http://www.cboe.com/products/vix-index-volatility/...
...vix-options-and-futures/vix-index/vix-historical-data"""

# File for vix from 2004 to present 'new' method of calculation
vix_file2004 = os.path.join('data','vixnew','vixcurrent.csv')
# File for vix from 1990 to end of 2003 'new' method of calculation
vix_file2003 = os.path.join('data','vixnew','vixarchive.xls')
vixcurrdf_raw = pd.read_csv(os.path.join(curr_dir, vix_file2004))
vixarchdf_raw = pd.read_excel(os.path.join(curr_dir, vix_file2003))
print('Current VIX DataFrame')
display(vixcurrdf_raw.info())
display(vixcurrdf_raw.isna().sum())
vixcurrdf_raw.head()

Current VIX DataFrame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4078 entries, 0 to 4077
Data columns (total 5 columns):
Cboe data is compiled for the convenience of site visitors and is furnished without responsibility for accuracy and is accepted by the site visitor on the condition that transmission or omissions shall not be made the basis for any claim demand or cause for action.  Your use of Cboe data is subject to the Terms and Conditions of Cboe Websites.    4078 non-null object
Unnamed: 1                                                                                                                                                                                                                                                                                                                                                   4078 non-null object
Unnamed: 2                                                                                                                          

None

Cboe data is compiled for the convenience of site visitors and is furnished without responsibility for accuracy and is accepted by the site visitor on the condition that transmission or omissions shall not be made the basis for any claim demand or cause for action.  Your use of Cboe data is subject to the Terms and Conditions of Cboe Websites.    0
Unnamed: 1                                                                                                                                                                                                                                                                                                                                                   0
Unnamed: 2                                                                                                                                                                                                                                                                                                

,Cboe data is compiled for the convenience of site visitors and is furnished without responsibility for accuracy and is accepted by the site visitor on the condition that transmission or omissions shall not be made the basis for any claim demand or cause for action. Your use of Cboe data is subject to the Terms and Conditions of Cboe Websites.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Date,VIX Open,VIX High,VIX Low,VIX Close
1,1/2/2004,17.96,18.68,17.54,18.22
2,1/5/2004,18.45,18.49,17.44,17.49
3,1/6/2004,17.66,17.67,16.19,16.73
4,1/7/2004,16.72,16.75,15.5,15.5


In [11]:
# Display archived DataFrame info and head
print('Archived VIX DataFrame')
display(vixarchdf_raw.info())
display(vixarchdf_raw.isna().sum())
vixarchdf_raw.head()

Archived VIX DataFrame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3533 entries, 0 to 3532
Data columns (total 5 columns):
CBOE data is compiled for the convenience of site visitors and is furnished without responsibility for accuracy and is accepted by the site visitor on the condition that transmission or omissions shall not be made the basis for any claim demand or cause for action.  Your use of CBOE data is subject to the Terms and Conditions of CBOE's Websites.    3533 non-null object
Unnamed: 1                                                                                                                                                                                                                                                                                                                                                     3025 non-null object
Unnamed: 2                                                                                                                     

None

CBOE data is compiled for the convenience of site visitors and is furnished without responsibility for accuracy and is accepted by the site visitor on the condition that transmission or omissions shall not be made the basis for any claim demand or cause for action.  Your use of CBOE data is subject to the Terms and Conditions of CBOE's Websites.      0
Unnamed: 1                                                                                                                                                                                                                                                                                                                                                     508
Unnamed: 2                                                                                                                                                                                                                                                                                        

,CBOE data is compiled for the convenience of site visitors and is furnished without responsibility for accuracy and is accepted by the site visitor on the condition that transmission or omissions shall not be made the basis for any claim demand or cause for action. Your use of CBOE data is subject to the Terms and Conditions of CBOE's Websites.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Date,VIX Open,VIX High,VIX Low,VIX Close
1,1990-01-02 00:00:00,NaN,NaN,NaN,17.24
2,1990-01-03 00:00:00,NaN,NaN,NaN,18.19
3,1990-01-04 00:00:00,NaN,NaN,NaN,19.22
4,1990-01-05 00:00:00,NaN,NaN,NaN,20.11


## Combine and Clean Tables


In [12]:
'''Constants for section'''
KEEP_REDDIT = ['Date', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7',
       'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15',
       'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23',
       'Top24', 'Top25']
KEEP_VIX = ['Date','VIX Open','VIX High','VIX Low','VIX Close']

In [13]:
'''Reformat VIX and Combine and Calculate the Difference for All'''
# copy raw data to new df
vixcurrdf =  vixcurrdf_raw.copy()
vixarchdf = vixarchdf_raw.copy()

# make colulmn names first row and then drop first row
vixcurrdf.columns = vixcurrdf.iloc[0]
vixcurrdf.drop(index=0,axis=0,inplace=True)
vixarchdf.columns = vixarchdf.iloc[0]
vixarchdf.drop(index=0,axis=0,inplace=True)

# convert date to datetime object
vixcurrdf['Date'] = pd.to_datetime(vixcurrdf['Date'],format='%m/%d/%Y')
vixarchdf['Date'] = pd.to_datetime(vixarchdf['Date'],format='%m/%d/%Y')

# combine, sort by date and reset index
vixdf = vixarchdf.append(vixcurrdf)
#vixdf.reset_index()
vixdf = vixdf.sort_values(by='Date',axis=0,ascending=True)

# Drop Rows where VIX Close is NaN
idxDrop = vixdf[KEEP_VIX][vixdf['VIX Close'].isnull()].index
vixdf.drop(idxDrop,axis=0,inplace=True)

# reset index and convert columns other than date to nummeric
vixdf = vixdf.reset_index(drop=True)
for col in KEEP_VIX[1:]:
    vixdf[col] = vixdf[col].astype(float)


# Fill VIX open NaN with previous close
# Fill High nan with max(opn,close)
# fill low nan with min(opn,close)
vc_last = vixdf.iloc[0]['VIX Close']
wasSet = False
for i,row in vixdf.iterrows():
    if math.isnan(row['VIX Open']):
        row['VIX Open'] = vc_last
        wasSet = True
    if math.isnan(row['VIX High']):
        row['VIX High'] = max(row['VIX Open'],row['VIX Close'])
        wasSet = True
    if math.isnan(row['VIX Low']):
        row['VIX Low'] = min(row['VIX Open'],row['VIX Close'])
        wasSet = True
    # only fix the row if something was set to speed up execution 
    if wasSet:
        vixdf.iloc[i] = row
    vc_last = row['VIX Close']
    wasSet = False

# Add Difference columns for each metric
newCols = []
for col in KEEP_VIX[1:]:
    newCols.append(col+' Delta')
    vixdf[newCols[-1]] = vixdf[col].diff(periods=1)

# Drop NaN first row and reindex, again
vixdf.dropna(axis=0, how='any',inplace=True)
vixdf.reset_index(drop=True,inplace=True)
display('NaN Tally',vixdf.isna().sum())
display(vixdf.info())
display(vixdf.head())
display(vixdf.tail())

'NaN Tally'

0
Date               0
VIX Open           0
VIX High           0
VIX Low            0
VIX Close          0
VIX Open Delta     0
VIX High Delta     0
VIX Low Delta      0
VIX Close Delta    0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7602 entries, 0 to 7601
Data columns (total 9 columns):
Date               7602 non-null datetime64[ns]
VIX Open           7602 non-null float64
VIX High           7602 non-null float64
VIX Low            7602 non-null float64
VIX Close          7602 non-null float64
VIX Open Delta     7602 non-null float64
VIX High Delta     7602 non-null float64
VIX Low Delta      7602 non-null float64
VIX Close Delta    7602 non-null float64
dtypes: datetime64[ns](1), float64(8)
memory usage: 534.6 KB


None

,Date,VIX Open,VIX High,VIX Low,VIX Close,VIX Open Delta,VIX High Delta,VIX Low Delta,VIX Close Delta
0,1990-01-03,17.24,18.19,17.24,18.19,0.00,0.95,0.00,0.95
1,1990-01-04,18.19,19.22,18.19,19.22,0.95,1.03,0.95,1.03
2,1990-01-05,19.22,20.11,19.22,20.11,1.03,0.89,1.03,0.89
3,1990-01-08,20.11,20.26,20.11,20.26,0.89,0.15,0.89,0.15
4,1990-01-09,20.26,22.20,20.26,22.20,0.15,1.94,0.15,1.94


,Date,VIX Open,VIX High,VIX Low,VIX Close,VIX Open Delta,VIX High Delta,VIX Low Delta,VIX Close Delta
7597,2020-03-09,41.94,62.12,41.94,54.46,0.48,7.73,1.10,12.52
7598,2020-03-10,49.68,55.66,43.56,47.30,7.74,-6.46,1.62,-7.16
7599,2020-03-11,52.24,55.82,49.98,53.90,2.56,0.16,6.42,6.60
7600,2020-03-12,61.46,76.83,59.91,75.47,9.22,21.01,9.93,21.57
7601,2020-03-13,71.31,77.57,55.17,57.83,9.85,0.74,-4.74,-17.64


In [13]:
'''Save or Retrive file for VIX Data'''
ENABLE_CELL = True   # must be true to execute save or retreive
SAVE = False           # dataframe if true retrieves if false
RETREIVE = 'vixdf20200322-124511.csv'

if ENABLE_CELL:
    if SAVE:
        vixdf_file = 'vixdf' + datetime.now().strftime("%Y%m%d-%H%M%S") + '.csv'
        vixdf.to_csv(os.path.join(curr_dir,'saves', vixdf_file))
        print('Files Saved to: ',vixdf_file)
    else:
        vixdf = pd.read_csv(os.path.join(curr_dir, 'saves',RETREIVE))
        print('File Retrieved: ',RETREIVE)
        vixdf.drop(columns=vixdf.columns[0],inplace=True)
    display(vixdf.head())

File Retrieved:  vixdf20200322-124511.csv


,Date,VIX Open,VIX High,VIX Low,VIX Close,VIX Open Delta,VIX High Delta,VIX Low Delta,VIX Close Delta
0,1990-01-03,17.24,18.19,17.24,18.19,0.00,0.95,0.00,0.95
1,1990-01-04,18.19,19.22,18.19,19.22,0.95,1.03,0.95,1.03
2,1990-01-05,19.22,20.11,19.22,20.11,1.03,0.89,1.03,0.89
3,1990-01-08,20.11,20.26,20.11,20.26,0.89,0.15,0.89,0.15
4,1990-01-09,20.26,22.20,20.26,22.20,0.15,1.94,0.15,1.94


In [15]:
'''Clean and Combine ABC and Reddit'''
# copy raw data to new df
abcdf =  abcdf_raw.copy()
redditdf = redditdf_raw.copy()

# Convert ot datetime
abcdf['publish_date'] = pd.to_datetime(abcdf['publish_date'],format='%Y%m%d')
redditdf['Date'] = pd.to_datetime(redditdf['Date'],format='%Y-%m-%d')

#combine, sort and reset index
cols = ['Date','ABC','Reddit']
abcdf.columns = cols[:2]
hl_df = pd.concat([abcdf,redditdf],sort=False,ignore_index=True)
hl_df = hl_df.sort_values(by='Date',axis=0,ascending=True)
hl_df = hl_df.reset_index(drop=True)
hl_df.fillna('',inplace=True)

display(hl_df.info())
display(hl_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188007 entries, 0 to 1188006
Data columns (total 28 columns):
Date     1188007 non-null datetime64[ns]
ABC      1188007 non-null object
Label    1188007 non-null object
Top1     1188007 non-null object
Top2     1188007 non-null object
Top3     1188007 non-null object
Top4     1188007 non-null object
Top5     1188007 non-null object
Top6     1188007 non-null object
Top7     1188007 non-null object
Top8     1188007 non-null object
Top9     1188007 non-null object
Top10    1188007 non-null object
Top11    1188007 non-null object
Top12    1188007 non-null object
Top13    1188007 non-null object
Top14    1188007 non-null object
Top15    1188007 non-null object
Top16    1188007 non-null object
Top17    1188007 non-null object
Top18    1188007 non-null object
Top19    1188007 non-null object
Top20    1188007 non-null object
Top21    1188007 non-null object
Top22    1188007 non-null object
Top23    1188007 non-null object
Top24    1188007 non-

None

,Date,ABC,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2003-02-19,aba decides against community broadcasting lic...,,,,,,,,,...,,,,,,,,,,
1,2003-02-19,patterson snubs health meeting to avoid lions den,,,,,,,,,...,,,,,,,,,,
2,2003-02-19,peace agreement may bring respite for venezuela,,,,,,,,,...,,,,,,,,,,
3,2003-02-19,pienaar shines as ajax frustrate arsenal,,,,,,,,,...,,,,,,,,,,
4,2003-02-19,plan for second skatepark,,,,,,,,,...,,,,,,,,,,


In [15]:
'''Save or Retrive file for Headline Data'''
ENABLE_CELL = True   # must be true to execute save or retreive
SAVE = False           # dataframe if true retrieves if false
RETREIVE = 'hl_df20200322-125057.csv'

if ENABLE_CELL:
    if SAVE:
        hl_df_file = 'hl_df' + datetime.now().strftime("%Y%m%d-%H%M%S") + '.csv'
        hl_df.to_csv(os.path.join(curr_dir,'saves', hl_df_file))
        print('File Saved to: ',hl_df_file)
    else:
        hl_df = pd.read_csv(os.path.join(curr_dir, 'saves',RETREIVE))
        print('File Retrieved: ',RETREIVE)
        hl_df.drop(columns=hl_df.columns[0],inplace=True)
        hl_df.fillna('',inplace=True)
        display(hl_df.head())

File Retrieved:  hl_df20200322-125057.csv


,Date,ABC,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2003-02-19,aba decides against community broadcasting lic...,,,,,,,,,...,,,,,,,,,,
1,2003-02-19,patterson snubs health meeting to avoid lions den,,,,,,,,,...,,,,,,,,,,
2,2003-02-19,peace agreement may bring respite for venezuela,,,,,,,,,...,,,,,,,,,,
3,2003-02-19,pienaar shines as ajax frustrate arsenal,,,,,,,,,...,,,,,,,,,,
4,2003-02-19,plan for second skatepark,,,,,,,,,...,,,,,,,,,,


## Score the Headlines

In [16]:
'''Score Function For Headline'''
import re

def scoreHeadline(sentdf,headline):
    #separate and capitalize words
    REG = r"[\w']+|[.,!?;]"
    keys = list(semantic_df.index)
    words = [x.upper() for x in re.findall(REG, headline) if x.isalpha()]
    
    # add up all the vectors of the words 
    # found in the headline and then find
    # the resulting unit vector
    total = np.array([0,0,0])
    for word in words:
        if word in keys:
            total = np.array(sentdf.loc[word]) + total
    norm = np.linalg.norm(total)
    return list(total/norm) if norm != 0 else list(total)


In [36]:
'''Find Set of Matching dates for data'''
matchingDates = hl_df.Date[hl_df.Date.isin(vixdf.Date)].unique()

abcCols = ['ABC Num Headlines','ABC Cummulative Sent','ABC Average Sent',
             'ABC Unit Vect Sent']
redditCols = ['Reddit Num Headlines','Reddit Cummulative Sent',
             'Reddit Average Sent','Reddit Unit Vect Sent']
totalCols = ['Num Headlines','Cummulative Sent',
             'Average Sent','Unit Vect Sent']
vixCols = ['VIX Open','VIX High','VIX Low','VIX Close','VIX Open Delta',
             'VIX High Delta','VIX Low Delta','VIX Close Delta']
finalCols = abcCols + redditCols + totalCols + vixCols

# build quick helper functions
avg = lambda c,s: list(s/c) if c != 0 else [0,0,0]
unit = lambda x: list(x/np.linalg.norm(x)) if np.linalg.norm(x) != 0 else [0,0,0]

final_df = pd.DataFrame(index = matchingDates, columns=finalCols)
for date in matchingDates:
    # Initialize Variables for looping
    cntABC = 0
    scoreABC = np.array([0,0,0])
    cntReddit = 0
    scoreReddit = np.array([0,0,0])
    
    # Get headlines from selected date
    hl = hl_df[hl_df.Date == date].drop(columns=['Date','Label']).reset_index(drop=True)
    
    # iterate through rows, score headline 
    for i,row in hl.iterrows():
        for j,col in enumerate(row):
            if col != '':
                if hl.columns[j] == 'ABC':
                    # count ABC headlines
                    cntABC = cntABC + 1
                    scoreABC = scoreHeadline(semantic_df,col) + scoreABC
                else:
                    # must be a reddit headline
                    cntReddit = cntReddit + 1
                    scoreReddit = scoreHeadline(semantic_df,col) + scoreReddit
    
    # build row for final dataframe
    cntTotal = cntABC + cntReddit
    scoreTotal = scoreABC + scoreReddit
    d = {}
    vals = [cntABC, list(scoreABC), avg(cntABC,scoreABC),unit(scoreABC),
         cntReddit, list(scoreReddit), avg(cntReddit,scoreReddit),unit(scoreReddit),
         cntTotal, list(scoreTotal), avg(cntTotal,scoreTotal), unit(scoreTotal)]
    # add in vix data
    vals = vals + (vixdf[vixdf.Date == date].drop(columns=['Date'],axis=1).values[0]).tolist()
    # add to dataframe by building series and then appending 
    for k,col in enumerate(finalCols):
        final_df.loc[date][col] = vals[k]
print('!!! Processing Successful !!!')

!!! Processing Successful !!!


In [38]:
'''Save or Retrive file for Headline Data'''
ENABLE_CELL = True   # must be true to execute save or retreive
SAVE = True           # dataframe if true retrieves if false
RETREIVE = 'final_df20200324-153407.csv'

if ENABLE_CELL:
    if SAVE:
        final_df_file = 'final_df' + datetime.now().strftime("%Y%m%d-%H%M%S") + '.csv'
        final_df.to_csv(os.path.join(curr_dir,'saves', final_df_file))
        print('File Saved to: ',final_df_file)
    else:
        xfinal_df = pd.read_csv(os.path.join(curr_dir, 'saves',RETREIVE))
        print('File Retrieved: ',RETREIVE)
        #final_df.drop(columns=hl_df.columns[0],inplace=True)
        #final_df.fillna('',inplace=True)
        display(xfinal_df.head())

File Saved to:  final_df20200324-153407.csv


In [39]:
final_df

,ABC Num Headlines,ABC Cummulative Sent,ABC Average Sent,ABC Unit Vect Sent,Reddit Num Headlines,Reddit Cummulative Sent,Reddit Average Sent,Reddit Unit Vect Sent,Num Headlines,Cummulative Sent,Average Sent,Unit Vect Sent,VIX Open,VIX High,VIX Low,VIX Close,VIX Open Delta,VIX High Delta,VIX Low Delta,VIX Close Delta
2003-02-19,198,"[-6.066885687491917, 78.46707772992892, 62.113...","[-0.030640836805514732, 0.3962983723733784, 0....","[-0.06051162271953582, 0.7826371631308697, 0.6...",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",198,"[-6.066885687491917, 78.46707772992892, 62.113...","[-0.030640836805514732, 0.3962983723733784, 0....","[-0.06051162271953582, 0.7826371631308697, 0.6...",31.17,31.72,31.13,31.31,-0.95,-0.4,0.02,0.2
2003-02-20,250,"[-33.766465096975764, 89.27560899994633, 79.42...","[-0.13506586038790305, 0.3571024359997853, 0.3...","[-0.27193633243828436, 0.7189761089269023, 0.6...",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",250,"[-33.766465096975764, 89.27560899994633, 79.42...","[-0.13506586038790305, 0.3571024359997853, 0.3...","[-0.27193633243828436, 0.7189761089269023, 0.6...",30.97,31.27,30.86,31.16,-0.2,-0.45,-0.27,-0.15
2003-02-21,250,"[-8.701007757573771, 88.18321489589258, 64.050...","[-0.034804031030295085, 0.3527328595835703, 0....","[-0.07958008149438293, 0.8065304184728874, 0.5...",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",250,"[-8.701007757573771, 88.18321489589258, 64.050...","[-0.034804031030295085, 0.3527328595835703, 0....","[-0.07958008149438293, 0.8065304184728874, 0.5...",30.98,31.65,30.16,30.25,0.01,0.38,-0.7,-0.91
2003-02-24,250,"[-3.1873345675980738, 81.84069224015485, 73.51...","[-0.012749338270392294, 0.3273627689606194, 0....","[-0.02896051615407678, 0.7436146533772695, 0.6...",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",250,"[-3.1873345675980738, 81.84069224015485, 73.51...","[-0.012749338270392294, 0.3273627689606194, 0....","[-0.02896051615407678, 0.7436146533772695, 0.6...",31.46,32.31,31.46,31.98,0.48,0.66,1.3,1.73
2003-02-25,250,"[-11.618003578377103, 89.06946135814738, 72.51...","[-0.04647201431350841, 0.3562778454325895, 0.2...","[-0.10063914054106567, 0.7715502908109586, 0.6...",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",250,"[-11.618003578377103, 89.06946135814738, 72.51...","[-0.04647201431350841, 0.3562778454325895, 0.2...","[-0.10063914054106567, 0.7715502908109586, 0.6...",32.71,33.06,31.6,31.74,1.25,0.75,0.14,-0.24
2003-02-26,250,"[-21.59189223797101, 91.23218236911634, 78.858...","[-0.08636756895188404, 0.3649287294764654, 0.3...","[-0.1762489387134877, 0.7447043150203827, 0.64...",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",250,"[-21.59189223797101, 91.23218236911634, 78.858...","[-0.08636756895188404, 0.3649287294764654, 0.3...","[-0.1762489387134877, 0.7447043150203827, 0.64...",32.12,32.21,31.75,31.94,-0.59,-0.85,0.15,0.2
2003-02-27,221,"[-2.878681554422996, 97.26892452383957, 59.565...","[-0.01302570839105428, 0.4401308801983691, 0.2...","[-0.025230644416348248, 0.8525283540483602, 0....",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",221,"[-2.878681554422996, 97.26892452383957, 59.565...","[-0.01302570839105428, 0.4401308801983691, 0.2...","[-0.025230644416348248, 0.8525283540483602, 0....",31.76,31.85,30.47,30.53,-0.36,-0.36,-1.28,-1.41
2003-02-28,249,"[-1.5707611266173356, 85.01356965637702, 62.40...","[-0.006308277616937091, 0.3414199584593454, 0....","[-0.014892749199464069, 0.8060332981184299, 0....",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",249,"[-1.5707611266173356, 85.01356965637702, 62.40...","[-0.006308277616937091, 0.3414199584593454, 0....","[-0.014892749199464069, 0.8060332981184299, 0....",29.84,29.84,29.06,29.63,-1.92,-2.01,-1.41,-0.9
2003-03-03,232,"[5.764656795499622, 83.5263475980158, 62.15774...","[0.024847658601291477, 0.360027360336275, 0.26...","[0.055282724695437185, 0.8010128343948175, 0.5...",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",232,"[5.764656795499622, 83.5263475980158, 62.15774...","[0.024847658601291477, 0.360027360336275, 0.26...","[0.055282724695437185, 0.8010128343948175, 0.5...